# Сultural heritage in Moscow
Мой знакомый спросил меня - как по твоему, какие районы города наиболее интересные с точки зрения объектов культурного наследия и при этом имеющин не самые высокие цены на квартиры?
В данном проекте мы попытаемся ответить на этот вопрос: оценить и классифицировать объекты культурного наследия (далее ОКН) в г.Москве, Россия. Определим наиболее интересные районы города с точки зрения оценок пользователей сервиса Foursquare и соотнесем эти данные со средней стоимостью квартир в том или  инном районе.
### Современная ситуация в Москве
На территории города Москвы расположено более 7 500 объектов, находящихся под государственной охраной.
Наибольшее число Объектов сконцентрировано в Центральном административном округе и самые высокие цены на недвижемость.
### А знаете ли вы, что?
На территории города Москвы расположено 3 объекта культурного наследия, включенных в Список всемирного культурного и природного наследия ЮНЕСКО:
* Ансамбль Кремля и Красная площадь
* Церковь Вознесения в Коломенском
* Ансамбль Новодевичьего монастыря

### Набор данных по ОКН находится на портале открытых данных правительства Москвы
Набор данных "Объекты культурного наследия города Москвы" содержит информацию об объектах, находящихся под государственной охраной, а именно: объектах культурного наследия, включенных в единый государственный реестр объектов культурного наследия (памятников истории и культуры) народов Российской Федерации и выявленных объектах культурного наследия.
#### Описание
https://data.mos.ru/opendata/7705021556-obekty-kulturnogo-naslediya-i-vyyavlennye-obekty-kulturnogo-naslediya/description?versionNumber=6&releaseNumber=19
#### Паcпорт
https://data.mos.ru/opendata/7705021556-obekty-kulturnogo-naslediya-i-vyyavlennye-obekty-kulturnogo-naslediya/passport?versionNumber=6&releaseNumber=19
#### Вход для разработчиков для получения ключа и чтения документации
https://apidata.mos.ru/
#### Структура данных
https://data.mos.ru/apiproxy/opendata/7705021556-obekty-kulturnogo-naslediya-i-vyyavlennye-obekty-kulturnogo-naslediya/structure-20200310(vs6).json

### Данные по стоимости недвижимости в районах

In [39]:
## -*- coding: utf-8 -*-
# !conda install -c conda-forge folium=0.5.0 --yes
# !conda install -c conda-forge geopy --yes
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows',500)
import json
import requests #
from pandas import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import folium # map rendering library
import lxml
import time
print('Done!')

Done!


In [5]:
url='https://data.mos.ru/apiproxy/opendata/7705021556-obekty-kulturnogo-naslediya-i-vyyavlennye-obekty-kulturnogo-naslediya/structure-20200310(vs6).json'
results = requests.get(url).json()

In [6]:
url ='https://apidata.mos.ru/version'
results = requests.get(url)
if results.ok:
    print('Version API is :{}'.format(results.json()['Version']))
else:
    print('"Houston, we have a problem!"----->', results)

Version API is :1


In [23]:
url='https://apidata.mos.ru/v1/datasets/530/rows?api_key=21d82d2344cd3f6e9deba56e6956cffa'
results = requests.get(url)

In [164]:
url='https://apidata.mos.ru/v1/datasets/530/rows?api_key=21d82d2344cd3f6e9deba56e6956cffa'
results = requests.get(url)
if results.ok:
    if results.json() == []: # Problem with data from server
        print('Problem with Dataset Online \nGet datas from offline')
        with open("Data.json", "r",encoding='utf-8') as read_file:
            df = json_normalize(json.load(read_file))
        print('It\'s all right!', df.shape)
    else:
        print('Get datas from online')
#       results.encoding = 'utf-8'
#       print(r.content.decode('utf-8'))
        df=json_normalize(results.json())
        df.drop(columns='global_id', inplace=True)
        # Rename columns as in offline file
        for i in df:
            df.rename(columns={i:i.replace('Cells.','')}, inplace=True)
        print('It\'s all right!', df.shape)
else:
    print('"Houston, we have a problem with server!"----->', results)
    # initialisation POI coordinate Series  

Get datas from online
It's all right! (8317, 15)


In [165]:
df2=df.copy()
df2.head()

,Number,global_id,AISID,ObjectNameOnDoc,ObjectName,EnsembleNameOnDoc,EnsembleName,AdmArea,District,SecurityStatus,Location,Category,ObjectType,geoData.coordinates,geoData.type
0,1,2949468,b249eae3-56f9-11e2-965f-005056806bb6,"- «Назаровская» башня, 1895 г., арх. К.М.Быков...",,Обсерватория Московского университета,,Центральный административный округ,Пресненский район,объект культурного наследия,"[{'Address': 'город Москва, Нововаганьковский ...",регионального значения,Сооружение,"[[[37.5689238433351, 55.7581989700618], [37.56...",Polygon
1,2,2949513,6c9899b0-5533-11e4-9457-005056806bb6,"- Жилой дом, 1908 г.",,"Жилые дома, 2-я половина XIX - начало XX вв.",,Центральный административный округ,район Замоскворечье,выявленный объект культурного наследия,"[{'Address': 'город Москва, улица Малая Ордынк...",категория не устанавливается,Здание,"[[[37.6258150552159, 55.7388775898882], [37.62...",Polygon
2,3,2949522,ff74c6e7-56f8-11e2-965f-005056806bb6,- Флигель,,"Службы городской усадьбы, XVIII - нач. XIX вв.",,Центральный административный округ,Басманный район,объект культурного наследия,"[{'Address': 'город Москва, Токмаков переулок,...",регионального значения,Здание,"[[[37.6708611603683, 55.7658349153926], [37.67...",Polygon
3,4,2949524,ff74c6ef-56f8-11e2-965f-005056806bb6,- Храм во имя св. блг. князя Александра Невско...,,Ансамбль Екатерининской больницы у Петровских ...,,Центральный административный округ,Тверской район,выявленный объект культурного наследия,"[{'Address': 'город Москва, Страстной бульвар,...",категория не устанавливается,Здание,"[[[37.610360655548, 55.7689052229478], [37.610...",Polygon
4,5,2949525,ff74c6f1-56f8-11e2-965f-005056806bb6,Евангелическая больница для оказания помощи бе...,,,,Центральный административный округ,Таганский район,выявленный объект культурного наследия,"[{'Address': 'город Москва, переулок Обуха, до...",категория не устанавливается,Здание,"[[[37.6541484601168, 55.7531461667182], [37.65...",Polygon


In [159]:
# import geopandas as gpd
# gdf=gpd.GeoDataFrame.from_features(results)

In [166]:
# get coordinate data
columns=['Lattitude','Longtitude']
new_df=pd.DataFrame(columns=columns)
for i in range(df.shape[0]):
    if df['geoData.type'][i]=='Polygon':
        new_df=new_df.append({'Lattitude':df['geoData.coordinates'][i][0][0][1],
                       'Longtitude':df['geoData.coordinates'][i][0][0][0]}, ignore_index=True)
    elif df['geoData.type'][i]=='MultiPolygon':
        new_df=new_df.append({'Lattitude':df['geoData.coordinates'][i][0][0][0][1],
                       'Longtitude':df['geoData.coordinates'][i][0][0][0][0]}, ignore_index=True)
    else:
        new_df=new_df.append({'Lattitude':df['geoData.coordinates'][i][1],
                       'Longtitude':df['geoData.coordinates'][i][0]}, ignore_index=True)
df[['Lattitude','Longtitude']]=new_df        

## MAPING

In [176]:
address = 'Moscow, Russia'
geolocator = Nominatim(user_agent="Moscow")
location = geolocator.geocode(address)
latitude,longitude = location.latitude,location.longitude
print(latitude,longitude)
Moscow_map=folium.Map(location=[latitude, longitude], zoom_start=11, tiles='OpenStreetMap')

55.7504461 37.6174943


In [168]:
df_lub=df[df['AdmArea'] == 'Юго-Восточный административный округ']
df_lub.reset_index(inplace=True)
df_lub.head()

,index,Number,global_id,AISID,ObjectNameOnDoc,ObjectName,EnsembleNameOnDoc,EnsembleName,AdmArea,District,SecurityStatus,Location,Category,ObjectType,geoData.coordinates,geoData.type,Lattitude,Longtitude
0,6,7,2949529,ff74c6f9-56f8-11e2-965f-005056806bb6,"Жилой поселок «Дангауэровка», 1927-1930 гг., а...",,,,Юго-Восточный административный округ,район Лефортово,выявленный объект культурного наследия,"[{'Address': 'город Москва, Авиамоторная улица...",категория не устанавливается,Комплекс,"[[[[37.7189925994458, 55.749565577948], [37.71...",MultiPolygon,55.749566,37.718993
1,39,40,2949622,80d5e314-56f9-11e2-965f-005056806bb6,"- Жилые корпуса: южный, северный, западный, во...",,Комплекс зданий Николо-Перервинского монастыря...,,Юго-Восточный административный округ,район Печатники,объект культурного наследия,"[{'Address': 'город Москва, Шоссейная улица, д...",федерального значения,Здание,"[[[37.7194042195947, 55.6695944842771], [37.71...",Polygon,55.669594,37.719404
2,88,89,2949762,80d5e3a5-56f9-11e2-965f-005056806bb6,"- Жилой дом, 1930 г., арх. Н.Молоков",,"Жилой поселок «Дангауэровка», 1927-1930 гг., а...",,Юго-Восточный административный округ,район Лефортово,выявленный объект культурного наследия,"[{'Address': 'город Москва, Авиамоторная улица...",категория не устанавливается,Здание,"[[[37.718096598416, 55.7494405650999], [37.718...",Polygon,55.749441,37.718097
3,105,106,2949812,80d5e3d7-56f9-11e2-965f-005056806bb6,"- Козина палата (Приют детский), середина XIX ...",,"Ансамбль Рогожской старообрядческой общины, XI...",,Юго-Восточный административный округ,Нижегородский район,объект культурного наследия,"[{'Address': 'город Москва, улица Рогожский По...",регионального значения,Здание,"[[[37.7024192664482, 55.7381192720904], [37.70...",Polygon,55.738119,37.702419
4,152,153,2949939,80d5e456-56f9-11e2-965f-005056806bb6,"- Жилой дом, 1928 г., арх. Е.Шервинский",,"Жилой поселок «Дангауэровка», 1927-1930 гг., а...",,Юго-Восточный административный округ,район Лефортово,выявленный объект культурного наследия,"[{'Address': 'город Москва, улица Пруд-Ключики...",категория не устанавливается,Здание,"[[[37.7203576372715, 55.7498570378526], [37.72...",Polygon,55.749857,37.720358


In [169]:
for i in range(df_lub.shape[0]):
    html = 'Place: {}'.format(df_lub.ObjectNameOnDoc[i])
    iframe = folium.IFrame(html=html, width=200, height=100)
    label = folium.Popup(iframe)
    latitude = df_lub.Lattitude[i]
    longtitude = df_lub.Longtitude[i]
    folium.Marker((latitude,longtitude), popup=label, icon=folium.Icon(color='red', icon='info-sign')).add_to(Moscow_map)
Moscow_map    

## Create K-Means Model and draw segments on map

In [170]:
kclusters = 9
# toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)
Moscow_clustering=df_lub[['global_id','District','SecurityStatus','Category','ObjectType','geoData.type','Lattitude','Longtitude']]
Moscow_clustering_new = pd.get_dummies(Moscow_clustering) 
# run k-means clustering
# kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Moscow_clustering_new.drop(columns=['global_id']))
# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([6, 4, 6, 0, 6, 0, 6, 4, 4, 4, 6, 0, 0, 4, 4, 4, 4, 4, 0, 4, 4, 4,
       6, 4, 6, 4, 6, 4, 0, 0, 6, 0, 0, 4, 0, 0, 6, 0, 4, 4, 0, 4, 4, 0,
       0, 4, 4, 4, 6, 0, 6, 6, 4, 4, 4, 4, 4, 4, 4, 6, 4, 4, 4, 0, 4, 4,
       6, 6, 4, 0, 4, 6, 4, 0, 4, 4, 4, 4, 4, 6, 4, 6, 4, 6, 4, 6, 4, 0,
       4, 0, 4, 0, 0, 4, 6, 6, 6, 4, 6, 4, 0, 4, 6, 6, 0, 0, 6, 0, 4, 4,
       0, 0, 0, 0, 7, 0, 0, 7, 4, 6, 0, 4, 4, 2, 4, 4, 4, 2, 2, 1, 1, 1,
       3, 2, 7, 1, 8, 3, 1, 0, 2, 8, 2, 0, 1, 3, 1, 2, 3, 1, 1, 8, 3, 7,
       3, 3, 3, 0, 8, 0, 7, 3, 1, 7, 1, 2, 1, 1, 1, 2, 4, 1, 0, 4, 1, 1,
       1, 4, 6, 1, 2, 3, 3, 1, 3, 8, 6, 2, 1, 1, 3, 2, 8, 8, 1, 2, 1, 1,
       1, 1, 8, 3, 8, 3, 8, 8, 3, 3, 3, 8, 8, 0, 1, 1, 1, 3, 2, 1, 1, 7,
       3, 0, 8, 1, 1, 3, 8, 2, 1, 4, 7, 3, 7, 3, 7, 1, 1, 3, 8, 2, 3, 3,
       1, 1, 3, 1, 2, 0, 1, 4, 4, 0, 1, 7, 2, 3, 1, 3, 3, 3, 3, 3, 7, 2,
       7, 1, 2, 1, 3, 7, 7, 4, 7, 1, 1, 3, 0, 8, 0, 1, 8, 1, 1, 3, 6, 0,
       1, 3, 1, 7, 1, 0, 1, 4, 2, 7, 0, 1, 3, 2, 1,

In [175]:
df_lub.head()

,Cluster Labels,index,Number,global_id,AISID,ObjectNameOnDoc,ObjectName,EnsembleNameOnDoc,EnsembleName,AdmArea,District,SecurityStatus,Location,Category,ObjectType,geoData.coordinates,geoData.type,Lattitude,Longtitude
0,6,6,7,2949529,ff74c6f9-56f8-11e2-965f-005056806bb6,"Жилой поселок «Дангауэровка», 1927-1930 гг., а...",,,,Юго-Восточный административный округ,район Лефортово,выявленный объект культурного наследия,"[{'Address': 'город Москва, Авиамоторная улица...",категория не устанавливается,Комплекс,"[[[[37.7189925994458, 55.749565577948], [37.71...",MultiPolygon,55.749566,37.718993
1,4,39,40,2949622,80d5e314-56f9-11e2-965f-005056806bb6,"- Жилые корпуса: южный, северный, западный, во...",,Комплекс зданий Николо-Перервинского монастыря...,,Юго-Восточный административный округ,район Печатники,объект культурного наследия,"[{'Address': 'город Москва, Шоссейная улица, д...",федерального значения,Здание,"[[[37.7194042195947, 55.6695944842771], [37.71...",Polygon,55.669594,37.719404
2,6,88,89,2949762,80d5e3a5-56f9-11e2-965f-005056806bb6,"- Жилой дом, 1930 г., арх. Н.Молоков",,"Жилой поселок «Дангауэровка», 1927-1930 гг., а...",,Юго-Восточный административный округ,район Лефортово,выявленный объект культурного наследия,"[{'Address': 'город Москва, Авиамоторная улица...",категория не устанавливается,Здание,"[[[37.718096598416, 55.7494405650999], [37.718...",Polygon,55.749441,37.718097
3,0,105,106,2949812,80d5e3d7-56f9-11e2-965f-005056806bb6,"- Козина палата (Приют детский), середина XIX ...",,"Ансамбль Рогожской старообрядческой общины, XI...",,Юго-Восточный административный округ,Нижегородский район,объект культурного наследия,"[{'Address': 'город Москва, улица Рогожский По...",регионального значения,Здание,"[[[37.7024192664482, 55.7381192720904], [37.70...",Polygon,55.738119,37.702419
4,6,152,153,2949939,80d5e456-56f9-11e2-965f-005056806bb6,"- Жилой дом, 1928 г., арх. Е.Шервинский",,"Жилой поселок «Дангауэровка», 1927-1930 гг., а...",,Юго-Восточный административный округ,район Лефортово,выявленный объект культурного наследия,"[{'Address': 'город Москва, улица Пруд-Ключики...",категория не устанавливается,Здание,"[[[37.7203576372715, 55.7498570378526], [37.72...",Polygon,55.749857,37.720358


In [174]:
# add clustering labels
df_lub.insert(0, 'Cluster Labels', kmeans.labels_)

In [179]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_lub['Lattitude'], df_lub['Longtitude'], df_lub['ObjectNameOnDoc'], df_lub['Cluster Labels']):
    label = folium.Popup(str(poi) + ' \nCluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.9).add_to(Moscow_map)
Moscow_map

In [ ]:
for i in range (0,kclusters):
    print('The Cluster is {}\nThe count of POI for this cluster is {} \nThe popular places in this points are:\n\n{}\n\n'.format(i,toronto_merged[toronto_merged['Cluster Labels'] == i].Borough.count(), 
                              toronto_merged[toronto_merged['Cluster Labels'] == i]['1st Most Common Venue'].value_counts(normalize=True).head(5)))
    print(toronto_merged.loc[toronto_merged['Cluster Labels'] == i, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]-9))]].head(5),'\n\n')    

In [622]:
CLIENT_ID = 'HBVBLMPIT2KIUOFH3P5LBXXZBZK0XG4WDXQB34WNEHJ3YJRB' # your Foursquare ID
CLIENT_SECRET = 'A1FGW51M2O3NWVLBQSMXMW4RJXHQY12J1IPDEYOJVUD0JNDB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 5
radius=30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HBVBLMPIT2KIUOFH3P5LBXXZBZK0XG4WDXQB34WNEHJ3YJRB
CLIENT_SECRET:A1FGW51M2O3NWVLBQSMXMW4RJXHQY12J1IPDEYOJVUD0JNDB


In [623]:
address = 'город Москва, Кутузовский проспект, 27'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

55.7417149 37.5314882


In [624]:
url_4sq ='https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, latitude, longitude,  radius, LIMIT)
print(url_4sq)

https://api.foursquare.com/v2/venues/explore?client_id=HBVBLMPIT2KIUOFH3P5LBXXZBZK0XG4WDXQB34WNEHJ3YJRB&client_secret=A1FGW51M2O3NWVLBQSMXMW4RJXHQY12J1IPDEYOJVUD0JNDB&v=20180604&ll=55.7417149,37.5314882&radius=30&limit=5


In [625]:
results_4sq = requests.get(url_4sq).json()
results_4sq['response']['totalResults']

1

In [626]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [627]:
venues = results_4sq['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng','venue.id']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng,id
0,Банзай,Sushi Restaurant,55.74146,37.53148,5a72efbadd70c575a6690d42


In [613]:
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '58b853ff9435a90a746443be',
   'name': 'Спортзал в Sberbank Agile Home',
   'location': {'lat': 55.74155328079935,
    'lng': 37.53104548803705,
    'labeledLatLngs': [{'label': 'display',
      'lat': 55.74155328079935,
      'lng': 37.53104548803705}],
    'distance': 33,
    'postalCode': '121170',
    'cc': 'RU',
    'city': 'Москва',
    'state': 'Москва',
    'country': 'Россия',
    'formattedAddress': ['121170, Москва', 'Россия']},
   'categories': [{'id': '4bf58dd8d48988d175941735',
     'name': 'Gym / Fitness Center',
     'pluralName': 'Gyms or Fitness Centers',
     'shortName': 'Gym / Fitness',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/gym_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-58b853ff9435a90a746443be-0'